# Stage 1 : Spectral peaks

In [16]:
import matplotlib.pyplot as plt
import librosa.display

import numpy as np
import pandas as pd
import librosa

In [17]:
def find_spectral_peaks(filename):
    y, sr = librosa.load(filename)
    D = np.abs(librosa.stft(y))
    tst_array = D
    max_filter_size = [91, 65]
    for x_init in range(0, tst_array.shape[0], max_filter_size[0]):
        for y_init in range(0, tst_array.shape[1], max_filter_size[1]):
            subarr = tst_array[x_init:(x_init + max_filter_size[0]), y_init:(y_init + max_filter_size[1])]
            mx = np.amax(subarr)
            tst_array[x_init:(x_init + max_filter_size[0]), y_init:(y_init + max_filter_size[1])] = np.where(subarr != mx, 0, subarr)
    # print("MAX FILTER APPLIED:")
    min_filter_size = [3, 3]
    for x_init in range(0, tst_array.shape[0], min_filter_size[0]):
        for y_init in range(0, tst_array.shape[1], min_filter_size[1]):
            subarr = tst_array[x_init:(x_init + min_filter_size[0]), y_init:(y_init + min_filter_size[1])]
            mn = np.amin(subarr)
            tst_array[x_init:(x_init + min_filter_size[0]), y_init:(y_init + min_filter_size[1])] = np.where(subarr != mn, subarr, 0)
    # print("MIN FILTER APPLIED:")
    return np.count_nonzero(tst_array), D

# Stage 2 : Storing hashes

In [18]:
import sqlite3
from rtree import index

In [19]:
conn = sqlite3.connect('quad.db')
print("Opened database successfully")

Opened database successfully


In [20]:
conn.execute('''CREATE TABLE UNNORMALIZED_QUADS
         (ID INT PRIMARY KEY     NOT NULL,
         AX  INT                 NOT NULL,
         AY  INT                 NOT NULL,
         BX  INT                 NOT NULL,
         BY  INT                 NOT NULL,
         CX  INT                 NOT NULL,
         CY  INT                 NOT NULL,
         DX  INT                 NOT NULL,
         DY  INT                 NOT NULL);''')


OperationalError: table UNNORMALIZED_QUADS already exists

In [21]:
conn.execute('''CREATE TABLE FIDINDEX
         (FID INT PRIMARY KEY    NOT NULL,
         FILENAME TEXT           NOT NULL,
         QUADIDX_START INT       NOT NULL,
         QUADIDX_END   INT       NOT NULL);''')

OperationalError: table FIDINDEX already exists

# System simulations

In [25]:
import os

path = '/home/supraraj/Projects/AudioFinger/audio_files/'

audio_input_files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.mp3' in file:
            audio_input_files.append(os.path.join(r, file))

In [27]:
for file in audio_input_files:
    print("")
    print("File: {}", file)
    peak_count, peaks = find_spectral_peaks(file)
    print("Peak count: {}", peak_count)


File: {} /home/supraraj/Projects/AudioFinger/audio_files/Nico & Vinz - Am I Wrong [Official Music Video].mp3
Peak count: {} 2388

File: {} /home/supraraj/Projects/AudioFinger/audio_files/P!nk - Just Give Me A Reason ft. Nate Ruess (1).mp3


KeyboardInterrupt: 